In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# size of the window for data preparation
train_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - train_window_size, train_window_size + 1):
    x_train_temp = x_data.iloc[i:i+train_window_size+1]
    y_train_temp = y_data.iloc[i:i+train_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [34]:
import itertools

# Define the hyperparameters to search over
input_sizes = [7]  # Example values, you can modify as needed
hidden_sizes = [64]  # Example values, you can modify as needed
num_layers_list = [ 4]  # Example values, you can modify as needed
learning_rates = [0.01, 0.001, 0.0001]  # Example values, you can modify as needed
window_sizes = [1, 2, 5, 10, 20, 30]  # Example values, you can modify as needed
num_epochs = 50  # Example value, you can modify as needed

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

best_validation_loss = float('inf')
best_hyperparameters = None

# Iterate over each hyperparameter combination
for hyperparameters in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparameters

    # Initialize the model with current hyperparameters
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validate the model
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        avg_train_loss = running_loss / len(x_train_tensor)
        avg_val_loss = val_loss / len(x_val_tensor)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')

    # Update best hyperparameters if validation loss is lower
    if avg_val_loss < best_validation_loss:
        best_validation_loss = avg_val_loss
        best_hyperparameters = hyperparameters

print("Best Hyperparameters:", best_hyperparameters)
print("Best Validation Loss:", best_validation_loss)


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 2802.71045218675, Validation Loss: 20511.021484375
Epoch [2/50], Training Loss: 2966.3470663110866, Validation Loss: 20511.306640625
Epoch [3/50], Training Loss: 2966.8633175046907, Validation Loss: 20511.306640625
Epoch [4/50], Training Loss: 2966.863611851763, Validation Loss: 20511.30859375
Epoch [5/50], Training Loss: 2966.863631219616, Validation Loss: 20511.30859375
Epoch [6/50], Training Loss: 2966.8638189292947, Validation Loss: 20511.30859375
Epoch [7/50], Training Loss: 2966.8637694747385, Validation Loss: 20511.30859375
Epoch [8/50], Training Loss: 2966.863661690744, Validation Loss: 20511.30859375
Epoch [9/50], Training Loss: 2966.8637959363723, Validation Loss: 20511.30859375
Epoch [10/50], Training Loss: 2966.8637582700662, Validation Loss: 20511.30859375
Epoch [11/50], Training Loss: 2966.8637786129993, Validation Loss: 20511.30859375
Epoch [12/50], Training Loss: 2966.8632451242215, Validation Loss: 20511.30859375
Epoch [13/50], Training Los

In [ ]:
p

In [ ]:
import torch
import torch.nn as nn

window_size = 5

input_size = 7
hidden_size = 64
num_layers = 4

learning_rate = 0.0001


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



Epoch [1/50], Training Loss: 21168.89639089087, Validation Loss: 21518.931640625
Epoch [2/50], Training Loss: 17636.3510741432, Validation Loss: 19394.4375
Epoch [3/50], Training Loss: 14376.367654258998, Validation Loss: 18585.720703125
Epoch [4/50], Training Loss: 12190.666216524925, Validation Loss: 18808.435546875


KeyboardInterrupt: 

In [ ]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 5

input_size = 7
hidden_size = 64
num_layers = 4

learning_rate = 0.0001
window_size = 1

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22321.16811415942, Validation Loss: 22740.544921875
Epoch [2/50], Training Loss: 19590.28790516201, Validation Loss: 20724.537109375
Epoch [3/50], Training Loss: 16547.792138645298, Validation Loss: 19086.052734375
Epoch [4/50], Training Loss: 14256.869639974313, Validation Loss: 17764.095703125
Epoch [5/50], Training Loss: 12470.339702296946, Validation Loss: 14572.294921875
Epoch [6/50], Training Loss: 11026.367790621085, Validation Loss: 12050.6611328125
Epoch [7/50], Training Loss: 9860.376608718745, Validation Loss: 10696.8740234375
Epoch [8/50], Training Loss: 8921.817034136271, Validation Loss: 9938.638671875
Epoch [9/50], Training Loss: 8043.265123274255, Validation Loss: 13117.8681640625
Epoch [10/50], Training Loss: 7257.726294857669, Validation Loss: 12189.380859375
Epoch [11/50], Training Loss: 6556.946482084052, Validation Loss: 11599.0166015625
Epoch [12/50], Training Loss: 5911.846699304278, Validation Loss: 10950.7451171875
Epoch [13/50], Tr

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22310.189176180917, Validation Loss: 22732.970703125
Epoch [2/50], Training Loss: 19489.045311181653, Validation Loss: 20659.318359375
Epoch [3/50], Training Loss: 16496.198450528394, Validation Loss: 19051.423828125
Epoch [4/50], Training Loss: 14212.207382488476, Validation Loss: 17554.0859375
Epoch [5/50], Training Loss: 12434.13509669828, Validation Loss: 14441.5048828125
Epoch [6/50], Training Loss: 10999.258462753673, Validation Loss: 12017.2333984375
Epoch [7/50], Training Loss: 9838.677909324168, Validation Loss: 10661.138671875
Epoch [8/50], Training Loss: 8904.555414873093, Validation Loss: 9883.8662109375
Epoch [9/50], Training Loss: 8021.146106980342, Validation Loss: 13021.037109375
Epoch [10/50], Training Loss: 7236.291375434951, Validation Loss: 11931.033203125
Epoch [11/50], Training Loss: 6535.871132014534, Validation Loss: 11254.1708984375
Epoch [12/50], Training Loss: 5891.500721726383, Validation Loss: 10501.0322265625
Epoch [13/50], Tra

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 1

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22298.065177423297, Validation Loss: 22724.119140625
Epoch [2/50], Training Loss: 19572.255441957874, Validation Loss: 20712.130859375
Epoch [3/50], Training Loss: 16550.06580962983, Validation Loss: 19092.041015625
Epoch [4/50], Training Loss: 14255.114293068717, Validation Loss: 17825.6796875
Epoch [5/50], Training Loss: 12468.671959173858, Validation Loss: 14470.8564453125
Epoch [6/50], Training Loss: 11023.952385422914, Validation Loss: 11996.7158203125
Epoch [7/50], Training Loss: 9857.388810463684, Validation Loss: 10708.4091796875


KeyboardInterrupt: 

In [ ]:
ä

Window size 10
learning_rate = 0.0001
 Convergence 18600

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 10

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 21168.212723235883, Validation Loss: 21479.0859375
Epoch [2/50], Training Loss: 19177.273087787467, Validation Loss: 20114.783203125
Epoch [3/50], Training Loss: 18658.154926966978, Validation Loss: 19643.70703125
Epoch [4/50], Training Loss: 18444.6954142026, Validation Loss: 19437.03125
Epoch [5/50], Training Loss: 18342.42485243402, Validation Loss: 19335.265625
Epoch [6/50], Training Loss: 18289.19282972164, Validation Loss: 19275.833984375
Epoch [7/50], Training Loss: 18231.6158119312, Validation Loss: 19234.19140625
Epoch [8/50], Training Loss: 16461.296068243097, Validation Loss: 18791.505859375
Epoch [9/50], Training Loss: 14737.73123033395, Validation Loss: 18652.333984375
Epoch [10/50], Training Loss: 12536.252306807568, Validation Loss: 19248.42578125
Epoch [11/50], Training Loss: 10880.909605747966, Validation Loss: 20499.564453125


KeyboardInterrupt: 

In [ ]:
ü

Window size 15
learning_rate = 0.0001
 Convergence 17500

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 15

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 21249.09915421254, Validation Loss: 19989.37109375
Epoch [2/50], Training Loss: 19155.82341264846, Validation Loss: 18688.98828125
Epoch [3/50], Training Loss: 18607.149545093358, Validation Loss: 18256.236328125
Epoch [4/50], Training Loss: 18374.904045453797, Validation Loss: 18070.513671875
Epoch [5/50], Training Loss: 18263.555876787665, Validation Loss: 17979.75390625
Epoch [6/50], Training Loss: 18191.043607627915, Validation Loss: 17928.45703125
Epoch [7/50], Training Loss: 18146.02200463909, Validation Loss: 17898.41015625
Epoch [8/50], Training Loss: 18112.174056383406, Validation Loss: 17880.982421875
Epoch [9/50], Training Loss: 18093.081012232782, Validation Loss: 17869.869140625
Epoch [10/50], Training Loss: 18044.843986290703, Validation Loss: 17856.98828125
Epoch [11/50], Training Loss: 16385.270544920204, Validation Loss: 17578.6015625
Epoch [12/50], Training Loss: 13627.171433671137, Validation Loss: 17770.515625


KeyboardInterrupt: 

In [ ]:
ä

Window = 20
learning_rate = 0.0001
"plateauing" or "convergence to a local minimum" 17400

smaller learning rate can help the model converge more smoothly, especially when dealing with longer sequences

increasing the model's capacity by adding more LSTM layers or increasing the number of hidden units in each layer. However, be cautious with this approach as it could lead to overfitting, especially with limited training data.

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 20

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 21230.766110158947, Validation Loss: 19784.11328125
Epoch [2/50], Training Loss: 19177.205511811386, Validation Loss: 18512.88671875
Epoch [3/50], Training Loss: 18626.71945567123, Validation Loss: 18092.296875
Epoch [4/50], Training Loss: 18395.354398610387, Validation Loss: 17913.35546875
Epoch [5/50], Training Loss: 18275.8739089434, Validation Loss: 17824.01953125
Epoch [6/50], Training Loss: 18208.995978844436, Validation Loss: 17775.4453125
Epoch [7/50], Training Loss: 18166.11870136649, Validation Loss: 17747.283203125
Epoch [8/50], Training Loss: 18135.667118678233, Validation Loss: 17729.71484375
Epoch [9/50], Training Loss: 15951.151754532646, Validation Loss: 17409.03515625
Epoch [10/50], Training Loss: 14231.196435787828, Validation Loss: 17631.70703125
Epoch [11/50], Training Loss: 17639.29298351449, Validation Loss: 17410.638671875
Epoch [12/50], Training Loss: 17625.646086827448, Validation Loss: 17412.48828125
Epoch [13/50], Training Loss: 1

KeyboardInterrupt: 